In [1]:
import pandas as pd
import json
import numpy as np
from pathlib import Path

In [2]:
##### Loading datasets -----

dat_birth = pd.read_csv('...')
dat_death = pd.read_csv('...')
dat_pop_o = pd.read_csv('...')
dat_pop_f = pd.read_csv('...')

In [ ]:
##### Some data cleaning -----

In [4]:
selectors = '' # to fill

fil_birth = dat_birth.query(selectors) 
fil_death = dat_death.query(selectors) 
fil_pop_o = dat_pop_o.query(selectors) 
fil_pop_f = dat_pop_f.query(selectors) 
fil_pop = pd.concat([fil_pop_o, fil_pop_f])

fil_birth.shape, fil_death.shape, fil_pop_o.shape, fil_pop_f.shape, fil_pop.shape

((42210, 12), (397980, 21), (1421070, 12), (1644381, 12), (3065451, 12))

In [5]:
locations = list(set(fil_death.Location))
locations.sort()

agespan = [1, 4] + [5] * 19 + [1]

In [36]:
for loc in locations:
    
    ##### Population data -----
    pop = fil_pop.query('Location == "{}"'.format(loc))
    pop_f = pop.groupby(['Time', 'AgeGrp']).PopFemale.sum().unstack() * 1e3
    pop_m = pop.groupby(['Time', 'AgeGrp']).PopMale.sum().unstack() * 1e3
    pop_t = pop.groupby(['Time', 'AgeGrp']).PopTotal.sum().unstack() * 1e3
    
    ##### Birth data -----
    bir = fil_birth.query('Location == "{}"'.format(loc))

    br = np.array(bir.groupby(['Time', 'AgeGrp']).ASFR.sum().unstack() / 1e3)
    br = np.repeat(np.repeat(br, 5, 0), 5, 1) 

    bbf = pop.query('15 <= AgeGrp < 50 and Time < 2100').groupby(['Time', 'AgeGrp']).PopFemale.sum()
    bbf = bbf.unstack() * 1e3

    bn = pd.DataFrame({
        'Time': range(1950, 2100), 
        'Births': list((br * bbf).sum(1).round())
    })
    bn = bn.set_index('Time')
    
    ##### Death data -----
    dea = fil_death.query('Location == "{}"'.format(loc))
    
    dea.loc[:, 'AgeGrp'] = list(np.repeat(range(len(agespan)), 3)) * int(dea.shape[0] / 3 / len(agespan))
    
    dea_f = dea.query('Sex == "Female"')
    dea_f = dea_f.groupby(['Time', 'AgeGrp']).mx.mean().unstack()
    dea_f = np.repeat(np.repeat(np.array(dea_f), 5, 0), agespan, 1)
    dea_f = pd.DataFrame(dea_f)
    dea_f['Time'] = range(1950, 2100)
    dea_f = dea_f.set_index('Time')

    dea_m = dea.query('Sex == "Male"')
    dea_m = dea_m.groupby(['Time', 'AgeGrp']).mx.mean().unstack()
    dea_m = np.repeat(np.repeat(np.array(dea_m), 5, 0), agespan, 1)
    dea_m = pd.DataFrame(dea_m)
    dea_m['Time'] = range(1950, 2100)
    dea_m = dea_m.set_index('Time')

    dea_t = dea.query('Sex == "Total"')
    dea_t = dea_t.groupby(['Time', 'AgeGrp']).mx.mean().unstack()
    dea_t = np.repeat(np.repeat(np.array(dea_t), 5, 0), agespan, 1)
    dea_t = pd.DataFrame(dea_t)
    dea_t['Time'] = range(1950, 2100)
    dea_t = dea_t.set_index('Time')
    
    ##### Output -----
    folder = "Input/ByCountry/{}".format(loc)
    Path(folder).mkdir(parents=True, exist_ok=True)
    
    pop_f.to_csv('{}/{}'.format(folder, 'PopF.csv'))
    pop_m.to_csv('{}/{}'.format(folder, 'PopM.csv'))
    pop_t.to_csv('{}/{}'.format(folder, 'PopT.csv'))
    bn.to_csv('{}/{}'.format(folder, 'Births.csv'))
    dea_f.to_csv('{}/{}'.format(folder, 'DeaF.csv'))
    dea_m.to_csv('{}/{}'.format(folder, 'DeaM.csv'))
    dea_t.to_csv('{}/{}'.format(folder, 'DeaT.csv'))
    
    print(loc)

Afghanistan
Albania
Algeria
Angola
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia (Plurinational State of)
Bosnia and Herzegovina
Botswana
Brazil
Brunei Darussalam
Bulgaria
Burkina Faso
Burundi
Cabo Verde
Cambodia
Cameroon
Canada
Central African Republic
Chad
Channel Islands
Chile
China
China, Hong Kong SAR
China, Macao SAR
Colombia
Comoros
Congo
Costa Rica
Croatia
Cuba
Curaçao
Cyprus
Czechia
Côte d'Ivoire
Dem. People's Republic of Korea
Democratic Republic of the Congo
Denmark
Djibouti
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Fiji
Finland
France
French Guiana
French Polynesia
Gabon
Gambia
Georgia
Germany
Ghana
Greece
Grenada
Guadeloupe
Guam
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran (Islamic Republic of)
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kiribati
Kuw